# PP5 - ML Brain Tumor Detector

## Notebook 1 - Data Collection

### Objectives

* Fetch data from kaggle and prepare data for further processes.


### Inputs

* 


### Outputs

| **output**      |          |       |
|-----------------|----------|-------|
| **train/**      | no_tumor | tumor |
| **test/**       | no_tumor | tumor |
| **validation/** | no_tumor | tumor |


### Additional Comments

* Dataset: [Kaggle](https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri?select=Training)
* License: [MIT](https://www.mit.edu/~amini/LICENSE.md)

---

### Import packages

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\tobis\\Documents\\GitHub\\ml-brain-tumor-detection\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\tobis\\Documents\\GitHub\\ml-brain-tumor-detection'

## Get data from Kaggle

**Install Kaggle**

In [5]:
%pip install kaggle==1.6.8

Note: you may need to restart the kernel to use updated packages.


**Change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON**

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
os.chmod("kaggle.json", 0o600)

**Set the kaggle dataset and download it**

In [7]:
KaggleDatasetPath = "sartajbhuvaji/brain-tumor-classification-mri"
DestinationFolder = "input/"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}


  0%|          | 0.00/86.8M [00:00<?, ?B/s]
  1%|          | 1.00M/86.8M [00:00<00:55, 1.63MB/s]
  2%|▏         | 2.00M/86.8M [00:00<00:29, 2.96MB/s]
  5%|▍         | 4.00M/86.8M [00:00<00:15, 5.45MB/s]
  6%|▌         | 5.00M/86.8M [00:01<00:13, 6.39MB/s]
  8%|▊         | 7.00M/86.8M [00:01<00:10, 8.00MB/s]
 10%|█         | 9.00M/86.8M [00:01<00:10, 7.47MB/s]
 12%|█▏        | 10.0M/86.8M [00:01<00:10, 7.97MB/s]
 14%|█▍        | 12.0M/86.8M [00:01<00:08, 8.94MB/s]
 16%|█▌        | 14.0M/86.8M [00:02<00:07, 9.61MB/s]
 18%|█▊        | 16.0M/86.8M [00:02<00:07, 10.2MB/s]
 21%|██        | 18.0M/86.8M [00:02<00:06, 10.7MB/s]
 23%|██▎       | 20.0M/86.8M [00:02<00:06, 10.5MB/s]
 25%|██▌       | 22.0M/86.8M [00:02<00:06, 9.80MB/s]
 28%|██▊       | 24.0M/86.8M [00:03<00:07, 9.31MB/s]
 30%|██▉       | 26.0M/86.8M [00:03<00:06, 10.3MB/s]
 32%|███▏      | 28.0M/86.8M [00:03<00:05, 11.2MB/s]
 35%|███▍      | 30.0M/86.8M [00:03<00:05, 9.99MB/s]
 37%|███▋      | 32.0M/86.8M [00:03<00:05, 9.93MB/s]
 

**Unzip the file and delete the zip folder.**

In [8]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/brain-tumor-classification-mri.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/brain-tumor-classification-mri.zip')

---

## Prepare the Data

**Create folder structure**

In [9]:
import shutil

In [10]:
testing_folder = 'input/Testing'
training_folder = 'input/Training'
parent_folder = 'input'

In [11]:
def create_folder_structure(parent_folder):
    brain_mri_scans_folder = os.path.join(parent_folder, 'brain-mri-scans')
    os.makedirs(brain_mri_scans_folder, exist_ok=True)
    
    tumor_folder = os.path.join(brain_mri_scans_folder, 'tumor')
    no_tumor_folder = os.path.join(brain_mri_scans_folder, 'no_tumor')
    os.makedirs(tumor_folder, exist_ok=True)
    os.makedirs(no_tumor_folder, exist_ok=True)

create_folder_structure(parent_folder)

**Merge pre-split folders (no_tumor)**

In [12]:
def move_and_rename_no_tumor_images(testing_folder, training_folder, parent_folder):
    no_tumor_destination = os.path.join(parent_folder, 'brain-mri-scans', 'no_tumor')
    
    counter = 1
    
    def rename_file(file_path):
        nonlocal counter
        file_name, file_ext = os.path.splitext(file_path)
        new_file_name = f'no_tumor_{counter}{file_ext}'
        os.rename(file_path, os.path.join(os.path.dirname(file_path), new_file_name))
        counter += 1
    
    testing_no_tumor_folder = os.path.join(testing_folder, 'no_tumor')
    if os.path.exists(testing_no_tumor_folder):
        for file in os.listdir(testing_no_tumor_folder):
            source_file_path = os.path.join(testing_no_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file(os.path.join(no_tumor_destination, file))
    
    training_no_tumor_folder = os.path.join(training_folder, 'no_tumor')
    if os.path.exists(training_no_tumor_folder):
        for file in os.listdir(training_no_tumor_folder):
            source_file_path = os.path.join(training_no_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file(os.path.join(no_tumor_destination, file))

move_and_rename_no_tumor_images(testing_folder, training_folder, parent_folder)

**Merge pre-split folders (glioma_tumor)**

In [13]:
def move_and_rename_glioma_tumor_images(testing_folder, training_folder, parent_folder):
    no_tumor_destination = os.path.join(parent_folder, 'brain-mri-scans', 'tumor')
    
    counter = 1
    
    def rename_file_glioma(file_path):
        nonlocal counter
        file_name, file_ext = os.path.splitext(file_path)
        new_file_name = f'glioma_{counter}{file_ext}'
        os.rename(file_path, os.path.join(os.path.dirname(file_path), new_file_name))
        counter += 1
    
    testing_glioma_tumor_folder = os.path.join(testing_folder, 'glioma_tumor')
    if os.path.exists(testing_glioma_tumor_folder):
        for file in os.listdir(testing_glioma_tumor_folder):
            source_file_path = os.path.join(testing_glioma_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_glioma(os.path.join(no_tumor_destination, file))
    
    training_glioma_tumor_folder = os.path.join(training_folder, 'glioma_tumor')
    if os.path.exists(training_glioma_tumor_folder):
        for file in os.listdir(training_glioma_tumor_folder):
            source_file_path = os.path.join(training_glioma_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_glioma(os.path.join(no_tumor_destination, file))

move_and_rename_glioma_tumor_images(testing_folder, training_folder, parent_folder)

**Merge pre-split folders (meningioma_tumor)**

In [14]:
def move_and_rename_meningioma_tumor_images(testing_folder, training_folder, parent_folder):
    no_tumor_destination = os.path.join(parent_folder, 'brain-mri-scans', 'tumor')
    
    counter = 1
    
    def rename_file_meningioma(file_path):
        nonlocal counter
        file_name, file_ext = os.path.splitext(file_path)
        new_file_name = f'meningioma_{counter}{file_ext}'
        os.rename(file_path, os.path.join(os.path.dirname(file_path), new_file_name))
        counter += 1
    
    testing_meningioma_tumor_folder = os.path.join(testing_folder, 'meningioma_tumor')
    if os.path.exists(testing_meningioma_tumor_folder):
        for file in os.listdir(testing_meningioma_tumor_folder):
            source_file_path = os.path.join(testing_meningioma_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_meningioma(os.path.join(no_tumor_destination, file))
    
    training_meningioma_tumor_folder = os.path.join(training_folder, 'meningioma_tumor')
    if os.path.exists(training_meningioma_tumor_folder):
        for file in os.listdir(training_meningioma_tumor_folder):
            source_file_path = os.path.join(training_meningioma_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_meningioma(os.path.join(no_tumor_destination, file))

move_and_rename_meningioma_tumor_images(testing_folder, training_folder, parent_folder)

**Merge pre-split folders (pituitary_tumor)**

In [15]:
def move_and_rename_pituitary_tumor_images(testing_folder, training_folder, parent_folder):
    no_tumor_destination = os.path.join(parent_folder, 'brain-mri-scans', 'tumor')
    
    counter = 1
    
    def rename_file_pituitary(file_path):
        nonlocal counter
        file_name, file_ext = os.path.splitext(file_path)
        new_file_name = f'pituitary_{counter}{file_ext}'
        os.rename(file_path, os.path.join(os.path.dirname(file_path), new_file_name))
        counter += 1
    
    testing_pituitary_tumor_folder = os.path.join(testing_folder, 'pituitary_tumor')
    if os.path.exists(testing_pituitary_tumor_folder):
        for file in os.listdir(testing_pituitary_tumor_folder):
            source_file_path = os.path.join(testing_pituitary_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_pituitary(os.path.join(no_tumor_destination, file))
    
    training_pituitary_tumor_folder = os.path.join(training_folder, 'pituitary_tumor')
    if os.path.exists(training_pituitary_tumor_folder):
        for file in os.listdir(training_pituitary_tumor_folder):
            source_file_path = os.path.join(training_pituitary_tumor_folder, file)
            shutil.move(source_file_path, no_tumor_destination)
            rename_file_pituitary(os.path.join(no_tumor_destination, file))

move_and_rename_pituitary_tumor_images(testing_folder, training_folder, parent_folder)

**Delete existing "Testing" and "Training" folder from input**

In [16]:
def delete_testing_and_training_folders(testing_folder, training_folder):
    if os.path.exists(testing_folder):
        shutil.rmtree(testing_folder)
    if os.path.exists(training_folder):
        shutil.rmtree(training_folder)

delete_testing_and_training_folders(testing_folder, training_folder)

In [20]:
! del input/*.csv
! del -rf input/mri-brain-tumor

Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [19]:
os.listdir('input/brain-mri-scans')

['no_tumor', 'tumor']

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [17]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (2852421808.py, line 5)